In [26]:
import os, sys
import json
import random
import cv2

from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
# from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
# from sklearn.model_selection import GridSearchCV
# from sklearn.metrics import make_scorer

module_path = os.path.abspath(os.path.join("..", ".."))
sys.path.append(module_path)
from dataset_utils.config import DS_ROOT
from dataset_utils.utils import imshow_by_path, imshow_by_name

from dataset_utils.Dataset import Dataset
from dataset_utils.utils import extract_eyes_region_from_aligned_face, gray, eqhist

import data_provider

# ひとまず，瞬きを負例として扱う

In [38]:
ANT_PATH = "./annotations.json"
DATA_DIR = os.path.join(DS_ROOT, "aligned_with_68")
with open(ANT_PATH, "r") as fr:
    ant = json.load(fr)

blink = {}
for name, label in sorted(ant.items()):
    if label == "blink":
        blink[name] = True
        
invalid = {}
for name, label in sorted(ant.items()):
    if label == "invalid_target":
        invalid[name] = True

In [35]:
with open(os.path.join(DS_ROOT, "blink.json"),  "w") as fw:
    json.dump(blink, fw)

In [43]:
with open(os.path.join(DS_ROOT, "invalid.json"), "w") as fw:
    json.dump(invalid, fw)